<a href="https://colab.research.google.com/github/Aranagut/INFO5731/blob/main/Project_ComMethods.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd 
import numpy as np
import re


In [2]:
import nltk
nltk.download('stopwords')
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer,PorterStemmer
from nltk.corpus import stopwords
import re
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [3]:
from google.colab import files
upload = files.upload()

Saving Twitter_Data.csv to Twitter_Data.csv


In [4]:
data = pd.read_csv('Twitter_Data.csv')

In [5]:
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer() 
nltk.download('wordnet')
nltk.download('omw-1.4')



def preprocess(sentence):
    sentence=str(sentence)
    sentence = sentence.lower()
    sentence=sentence.replace('{html}',"") 
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', sentence)
    rem_url=re.sub(r'http\S+', '',cleantext)
    rem_num = re.sub('[0-9]+', '', rem_url)
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(rem_num)  
    filtered_words = [w for w in tokens if len(w) > 2 if not w in stopwords.words('english')]
    stem_words=[stemmer.stem(w) for w in filtered_words]
    lemma_words=[lemmatizer.lemmatize(w) for w in stem_words]
    return " ".join(filtered_words)

data['cleanText']=data['Tweets'].map(lambda s:preprocess(s)) 
data.head()

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


,Tweets,Location,Date,cleanText
0,"['state', '＃antonyblinken', 'fm', 'wang', 'ysa...","75-77 Malone Road, Belfast",2022-11-03 20:49:52+00:00,state antonyblinken wang ysaid bringing chinau...
1,"['😂🥰🥰🥰🥰🥰', 'china', 'korea', 'twittertakeover'...",NaN,2022-11-03 20:49:38+00:00,china korea twittertakeover viral doctor denta...
2,"['chinamade', 'flag', 'pins', 'even', 'mps', '...","Ontario, Canada",2022-11-03 20:49:35+00:00,chinamade flag pins even mps condemned china g...
3,"['inas', 'covid', 'restrictions', 'relaxed', '...",nature New Mexico USA,2022-11-03 20:49:34+00:00,inas covid restrictions relaxed weeks less dra...
4,"['seen', 'photograph', 'taken', 'hong', 'kong'...","Shenzhen, China",2022-11-03 20:49:27+00:00,seen photograph taken hong kong crest hill ret...


In [6]:
from textblob import TextBlob 
def get_tweet_sentiment(tweet): 

        analysis = TextBlob(tweet)
        # set sentiment 
        if analysis.sentiment.polarity > 0: 
            return 'positive'
        elif analysis.sentiment.polarity == 0: 
            return 'neutral'
        else: 
            return 'negative'

In [7]:
b=[]

for i in data['Tweets']:
    b.append(get_tweet_sentiment(i))

data['Sentiments']=b

In [8]:
data[data['Sentiments']=='negative']['Sentiments']=-1
data[data['Sentiments']=='neutral']['Sentiments']=0
data[data['Sentiments']=='positive']['Sentiments']=1

<ipython-input-8-dbfcc335132e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[data['Sentiments']=='negative']['Sentiments']=-1
<ipython-input-8-dbfcc335132e>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[data['Sentiments']=='neutral']['Sentiments']=0
<ipython-input-8-dbfcc335132e>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer 

tfidf_vectorizer = TfidfVectorizer(lowercase = False, analyzer='word')
tweets_tfidf = tfidf_vectorizer.fit_transform(data['cleanText']).toarray()

In [10]:
data.head()

,Tweets,Location,Date,cleanText,Sentiments
0,"['state', '＃antonyblinken', 'fm', 'wang', 'ysa...","75-77 Malone Road, Belfast",2022-11-03 20:49:52+00:00,state antonyblinken wang ysaid bringing chinau...,negative
1,"['😂🥰🥰🥰🥰🥰', 'china', 'korea', 'twittertakeover'...",NaN,2022-11-03 20:49:38+00:00,china korea twittertakeover viral doctor denta...,neutral
2,"['chinamade', 'flag', 'pins', 'even', 'mps', '...","Ontario, Canada",2022-11-03 20:49:35+00:00,chinamade flag pins even mps condemned china g...,neutral
3,"['inas', 'covid', 'restrictions', 'relaxed', '...",nature New Mexico USA,2022-11-03 20:49:34+00:00,inas covid restrictions relaxed weeks less dra...,negative
4,"['seen', 'photograph', 'taken', 'hong', 'kong'...","Shenzhen, China",2022-11-03 20:49:27+00:00,seen photograph taken hong kong crest hill ret...,positive


In [11]:
out = data.iloc[:, -1].values

In [12]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(tweets_tfidf, out, test_size = 0.3)

In [13]:
from sklearn import svm

classifier_svm = svm.SVC()
model_svm = classifier_svm.fit(x_train, y_train) 
svm_predictions_validation_set = classifier_svm.predict(x_test) 

In [14]:
from sklearn.metrics import accuracy_score
accuracy_score(svm_predictions_validation_set, y_test)

0.8908333333333334

In [15]:
classifier_svm.predict([x_test[0]])[0] 

'neutral'

In [16]:
!pip install aspect-based-sentiment-analysis

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 481 kB 7.7 MB/s 
     |████████████████████████████████| 516.3 MB 3.3 kB/s 
     |████████████████████████████████| 348 kB 67.8 MB/s 
     |████████████████████████████████| 98 kB 8.0 MB/s 
     |████████████████████████████████| 26.0 MB 10.9 MB/s 
     |████████████████████████████████| 3.0 MB 40.2 MB/s 
     |████████████████████████████████| 2.9 MB 34.5 MB/s 
     |████████████████████████████████| 454 kB 59.9 MB/s 
     |████████████████████████████████| 880 kB 45.1 MB/s 
     |████████████████████████████████| 132 kB 69.2 MB/s 
     |████████████████████████████████| 1.3 MB 62.3 MB/s 
     |████████████████████████████████| 7.6 MB 27.5 MB/s 
     |████████████████████████████████| 152 kB 63.8 MB/s 
     |████████████████████████████████| 10.2 MB 40.4 MB/s 
     |████████████████████████████████| 79 kB 9.3 MB/s 
     |███████████████████████████

In [17]:
import aspect_based_sentiment_analysis as ABSA

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [18]:
nlp = ABSA.load()

Downloading:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/438M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [19]:
text = ("We are great fans of Slack, but we wish the subscriptions "
        "were more accessible to small startups.")

slack, price = nlp(text, aspects=['slack', 'price'])
assert price.sentiment == ABSA.Sentiment.negative
assert slack.sentiment == ABSA.Sentiment.positive

In [20]:
text = data['cleanText'][:5].values

In [21]:
t=tuple(text)
str(t)

"('state antonyblinken wang ysaid bringing chinaus relations back track stable development meets common expectation world', 'china korea twittertakeover viral doctor dentalhealth', 'chinamade flag pins even mps condemned china genocide national symbol country cdnpoli pablorodriguez', 'inas covid restrictions relaxed weeks less draconian zero policies less quarantine time foreign travelers less constant daily testing boast world economy chinalockdown businessgrowth', 'seen photograph taken hong kong crest hill retaken lowu border crossing bottom right image geography urbangrowth urbanlife china shenzhen urbandevelopment')"

In [22]:
president, china, bank, health = nlp(str(t), aspects=['president', 'china', 'bank', 'health'])

In [23]:
president.sentiment

<Sentiment.neutral: 0>

In [24]:
china.sentiment

<Sentiment.neutral: 0>

In [25]:
bank.sentiment

<Sentiment.negative: 1>

In [26]:
health.sentiment

<Sentiment.neutral: 0>

In [27]:
text_data = str(data['cleanText'][:])

In [30]:
Economy, Health, Military, Education, Culture, Technology, Politics = nlp(text_data, aspects = ['Economy', 'Health', 'Army', 'Education', 'Culture', 'Technology','politics'])

In [31]:
ABSA.summary(Economy)

Sentiment.neutral for "Economy"
Scores (neutral/negative/positive): [0.579 0.334 0.087]


In [32]:
ABSA.summary(Health)

Sentiment.neutral for "Health"
Scores (neutral/negative/positive): [0.586 0.301 0.113]


In [34]:
ABSA.summary(Military)

Sentiment.neutral for "Army"
Scores (neutral/negative/positive): [0.574 0.354 0.072]


In [33]:
ABSA.summary(Education)

Sentiment.neutral for "Education"
Scores (neutral/negative/positive): [0.715 0.195 0.09 ]


In [35]:
ABSA.summary(Culture)

Sentiment.neutral for "Culture"
Scores (neutral/negative/positive): [0.705 0.219 0.076]


In [36]:
ABSA.summary(Technology)

Sentiment.negative for "Technology"
Scores (neutral/negative/positive): [0.41  0.478 0.112]


In [37]:
ABSA.summary(Politics)

Sentiment.neutral for "politics"
Scores (neutral/negative/positive): [0.711 0.193 0.096]
